# Extracting Authors order from Publications data

This Python notebook shows how to use the [Dimensions Analytics API](https://www.dimensions.ai/dimensions-apis/), in particular the [publications source](https://docs.dimensions.ai/dsl/datasource-publications.html), in order to analyse the publications' authors' order.

These are the steps:

* First we extract a dataset of interest from Dimensions' publications database
* Second, we process authors structured data so to turn the implicit authorship order into a number
* Third, we mark first and last authors via a new 'author category' column


In [1]:
import datetime
print("==\nCHANGELOG\nThis notebook was last run on %s\n==" % datetime.date.today().strftime('%b %d, %Y'))

==
CHANGELOG
This notebook was last run on Apr 20, 2023
==


## Prerequisites

This notebook assumes you have installed the [Dimcli](https://pypi.org/project/dimcli/) library and have followed the steps in the ['Getting Started' tutorial](https://api-lab.dimensions.ai/cookbooks/1-getting-started/1-Using-the-Dimcli-library-to-query-the-API.html).

In [2]:
!pip install dimcli plotly tqdm -U --quiet 

import dimcli
from dimcli.utils import *

import os, sys, time, json
from tqdm.notebook import tqdm as progressbar

import pandas as pd
import numpy as np

print("==\nLogging in..")
# https://digital-science.github.io/dimcli/getting-started.html#authentication
ENDPOINT = "https://app.dimensions.ai"
if 'google.colab' in sys.modules:
  import getpass
  KEY = getpass.getpass(prompt='API Key: ')  
  dimcli.login(key=KEY, endpoint=ENDPOINT)
else:
  KEY = ""
  dimcli.login(key=KEY, endpoint=ENDPOINT)
dsl = dimcli.Dsl()

Searching config file credentials for 'https://app.dimensions.ai' endpoint..


==
Logging in..
Dimcli - Dimensions API Client (v1.0.2)
Connected to: <https://app.dimensions.ai/api/dsl> - DSL v2.6
Method: dsl.ini file


## 1. Extracting a dataset from Dimensions 

We use three different queries in order to extract 

* authors information
* publications metadata
* research organizations information 


**NOTE** other approaches are also possible e.g. extracting all data via a single query and then using Python to select only the fields of interests. For the purpose of this tutorial, using separate queries is the most straighforward way to achieve our goal.


In [3]:
#
# the main query string selects publications based on a) pub year, b) specific organization IDs and c) concept
# you can update this query based on your preferences
#

main_query = """
search publications 
    where year in [2022:2022] 
    and research_orgs in ["grid.21925.3d","grid.147455.6","grid.25879.31","grid.29857.31"]
    and concepts = "oncology"
 return publications
"""

In [4]:
# use the main query but extract only authors infos
Authors = dsl.query_iterative(main_query + "[id+authors]").as_dataframe_authors()  ##researcher_id, pub_id, current_organization_ID
Authors.head()

Starting iteration with limit=1000 skip=0 ...
0-120 / 120 (4.36s)
===
Records extracted: 120


affiliations corresponding  \
0  [{'city': 'Philadelphia', 'city_id': 4560349, ...                 
1  [{'city': 'Jacksonville', 'city_id': 4160021, ...          True   
2  [{'city': 'Madison', 'city_id': 5261457, 'coun...                 
3  [{'city': 'Madison', 'city_id': 5261457, 'coun...                 
4  [{'city': 'Madison', 'city_id': 5261457, 'coun...                 

  current_organization_id  first_name   last_name                  orcid  \
0           grid.25879.31      Andrew    Schlafly                   None   
1           grid.25879.31      Ronnie       Sebro                   None   
2           grid.14003.36  Jessica R.  Schumacher  [0000-0002-6740-9498]   
3           grid.14003.36   Alyssa A.      Wiener                   None   
4           grid.410427.4  Caprice C.   Greenberg                   None   

                                     raw_affiliation       researcher_id  \
0  [Perelman School of Medicine, University of Pe...  ur.012676303143.43   
1  [Center for Augmented Intelligence, Mayo Clini...    ur.0660765735.77   
2  [Department of Surgery, University of Wisconsi...    ur.0661627033.29   
3  [Department of Surgery, University of Wisconsi...  ur.015612367333.32   
4  [Department of Surgery, University of Wisconsi...  ur.012326542557.13   

           pub_id  
0  pub.1154094821  
1  pub.1154094821  
2  pub.1153677611  
3  pub.1153677611  
4  pub.1153677611

In [5]:
# use the main query but extract only pubs metadata
Pubs = dsl.query_iterative(main_query + "[id+title+year+times_cited]").as_dataframe() 
Pubs.head()

Starting iteration with limit=1000 skip=0 ...
0-120 / 120 (1.88s)
===
Records extracted: 120


id                                              title  \
0  pub.1154094821  Does NIH funding differ between medical specia...   
1  pub.1153677611  Local/Regional Recurrence Rates After Breast-C...   
2  pub.1153575321  Quality and Safety Considerations in Intensity...   
3  pub.1153525111  Data standards in pediatric oncology: Past, pr...   
4  pub.1153522196  Assessments of Somatic Variant Classification ...   

   times_cited  year  
0            0  2022  
1            0  2022  
2            0  2022  
3            0  2022  
4            0  2022

In [6]:
# use the main query but extract only research orgs infos
RORGS = dsl.query_iterative(main_query + "[unnest(research_orgs)]").as_dataframe()
RORGS.head()

Starting iteration with limit=1000 skip=0 ...
0-120 / 120 (1.03s)
120-120 / 120 (4.07s)
===
Records extracted: 599


research_orgs.city_name research_orgs.country_name research_orgs.id  \
0            Philadelphia              United States    grid.25879.31   
1            Jacksonville              United States    grid.417467.7   
2                 Madison              United States    grid.14003.36   
3               Rochester              United States    grid.66875.3a   
4                 Madison              United States    grid.412639.b   

   research_orgs.latitude                              research_orgs.linkout  \
0               39.952457                            [http://www.upenn.edu/]   
1               30.289337  [https://www.mayoclinic.org/patient-visitor-gu...   
2               43.076694                             [http://www.wisc.edu/]   
3               44.024070  [http://www.mayoclinic.org/patient-visitor-gui...   
4               43.076946                         [https://cancer.wisc.edu/]   

   research_orgs.longitude               research_orgs.name  \
0               -75.193220       University of Pennsylvania   
1               -81.437775                      Mayo Clinic   
2               -89.412440  University of Wisconsin–Madison   
3               -92.466310                      Mayo Clinic   
4               -89.431470         UW Carbone Cancer Center   

  research_orgs.state_name research_orgs.types research_orgs.acronym  
0             Pennsylvania         [Education]                   NaN  
1                  Florida        [Healthcare]                   NaN  
2                Wisconsin         [Education]                    UW  
3                Minnesota        [Healthcare]                   NaN  
4                Wisconsin        [Healthcare]                 UWCCC

## 2. Combining the results 

We merge the results from the queries above into a single table containing only the columns we want. 

Additionally, we calculate for each author which is the order of authorship and add a category for 'first' and 'last' authors. 


In [7]:
#
# Authors becomes the "main table" because it has both the PubID and the ResearcherID
# Then use Authors->Pubs to lookup title, year, times cited      on authors.pub_id = Pubs.id
# Then use Authors->RORGS to lookup rorg name, type and country  on authors.current_organization_ID = RORGS.id 
#


##prep RORGS for merge

RORGS = RORGS.dropna(subset = ['research_orgs.id'])
RORGS = RORGS.rename(columns = {'research_orgs.id':'rorg_id'})
RORGS = RORGS.drop_duplicates(subset=['rorg_id', 'research_orgs.name'], keep='last')


##Combine all three dataframes into one

AutPub = pd.merge(
    left=Authors,
    right=Pubs,
    left_on='pub_id',
    right_on='id',
    how='left'
)

final = pd.merge(
    left=AutPub,
    right=RORGS,
    left_on='current_organization_id',
    right_on='rorg_id',
    how='left'
)

final["author_name"] = final["last_name"] + [", "] + final["first_name"]
final['author_number'] = final.groupby(['pub_id']).cumcount()+1;  #this will only work if you haven't sorted the dataframe
final = final.drop(columns=['affiliations', 'corresponding', 'raw_affiliation', 'id', 'first_name', 'last_name','research_orgs.latitude','research_orgs.longitude','research_orgs.acronym'])

#Get AuthorCounts,etc by pub ID and join back to AutPubRORG table

AuthorCount = final.groupby(['pub_id'])['author_number'].max()

final = pd.merge(
    left=final,
    right=AuthorCount,
    left_on='pub_id',
    right_on='pub_id',
    how='left'
)

final = final.rename(columns = {'author_number_x':'author_number', 'author_number_y':'authors_tot', })


# Assing a category to first authors and last authors 

final['AuthorCategory'] = np.where(
     final['author_number']==1, 'FirstAuthor', 
         np.where(
            final['author_number']==final['authors_tot'],"LastAuthor",
             np.where(
                (final['authors_tot']-final['author_number'])==1,"Penultimate",""
             )
         )
)

final.head(20)


current_organization_id                  orcid       researcher_id  \
0            grid.25879.31                   None  ur.012676303143.43   
1            grid.25879.31                   None    ur.0660765735.77   
2            grid.14003.36  [0000-0002-6740-9498]    ur.0661627033.29   
3            grid.14003.36                   None  ur.015612367333.32   
4            grid.410427.4                   None  ur.012326542557.13   
5            grid.14003.36  [0000-0002-4517-1204]    ur.0632670166.10   
6            grid.240614.5                   None    ur.0671641425.86   
7            grid.66875.3a                   None   ur.01264057027.05   
8            grid.65499.37  [0000-0002-4722-4824]  ur.012333143317.98   
9            grid.66875.3a  [0000-0002-2234-7430]    ur.0654547635.88   
10           grid.14003.36                   None  ur.016365762407.99   
11           grid.29857.31  [0000-0002-9790-2988]   ur.07542517775.28   
12           grid.14003.36                   None  ur.012527107034.48   
13           grid.14003.36  [0000-0001-8796-4328]   ur.01224567375.46   
14           grid.66875.3a                   None  ur.014436171657.16   
15           grid.14003.36                   None   ur.01333351663.72   
16                                             []                None   
17                                             []                None   
18           grid.478397.6                   None  ur.016200147053.28   
19           grid.478397.6                   None  ur.010477224250.27   

            pub_id                                              title  \
0   pub.1154094821  Does NIH funding differ between medical specia...   
1   pub.1154094821  Does NIH funding differ between medical specia...   
2   pub.1153677611  Local/Regional Recurrence Rates After Breast-C...   
3   pub.1153677611  Local/Regional Recurrence Rates After Breast-C...   
4   pub.1153677611  Local/Regional Recurrence Rates After Breast-C...   
5   pub.1153677611  Local/Regional Recurrence Rates After Breast-C...   
6   pub.1153677611  Local/Regional Recurrence Rates After Breast-C...   
7   pub.1153677611  Local/Regional Recurrence Rates After Breast-C...   
8   pub.1153677611  Local/Regional Recurrence Rates After Breast-C...   
9   pub.1153677611  Local/Regional Recurrence Rates After Breast-C...   
10  pub.1153677611  Local/Regional Recurrence Rates After Breast-C...   
11  pub.1153677611  Local/Regional Recurrence Rates After Breast-C...   
12  pub.1153677611  Local/Regional Recurrence Rates After Breast-C...   
13  pub.1153677611  Local/Regional Recurrence Rates After Breast-C...   
14  pub.1153677611  Local/Regional Recurrence Rates After Breast-C...   
15  pub.1153677611  Local/Regional Recurrence Rates After Breast-C...   
16  pub.1153575321  Quality and Safety Considerations in Intensity...   
17  pub.1153575321  Quality and Safety Considerations in Intensity...   
18  pub.1153575321  Quality and Safety Considerations in Intensity...   
19  pub.1153575321  Quality and Safety Considerations in Intensity...   

    times_cited  year research_orgs.city_name research_orgs.country_name  \
0             0  2022            Philadelphia              United States   
1             0  2022            Philadelphia              United States   
2             0  2022                 Madison              United States   
3             0  2022                 Madison              United States   
4             0  2022                 Augusta              United States   
5             0  2022                 Madison              United States   
6             0  2022                 Buffalo              United States   
7             0  2022               Rochester              United States   
8             0  2022                  Boston              United States   
9             0  2022               Rochester              United States   
10            0  2022                 Madison              United States   
11            0  2022  

---
## Where to go from here

In this [Dimensions Analytics API](https://www.dimensions.ai/dimensions-apis/) tutorial we have seen how, using the [publications source](https://docs.dimensions.ai/dsl/datasource-publications.html), it is possible to extract and analyse information about authors and their order to authorhip.

This only scratches the surface of the possible applications of publications data, but hopefully it'll give you a few basic tools to get started building your own application. 

For more tutorials, see the [API LAB homepage](https://api-lab.dimensions.ai/).